In [ ]:
import re
import pandas as pd
import geonamescache as gnc
from unidecode import unidecode


gc = gnc.GeonamesCache()
cities = [unidecode(city["name"]) for city in gc.get_cities().values()]
countries = [unidecode(country["name"]) for country in gc.get_countries().values()]

def read_file(filename):
    headlines = []
    with open(filename, 'r') as file:
        for line in file.readlines():
            headlines.append(unidecode(line).rstrip("\n"))
            
    return headlines

def parse_data(headlines):
    rows = {}
    
    for city in cities:
        regexp = re.compile(".*({})( |-|'|\?|,|$)".format(city))
        for line in headlines:
            if re.match(regexp, line):
                rows[line]={"city":city}

    for country in countries:
        regexp = re.compile(".*({})( |-|'|\?|,|$)".format(country))
        for line in headlines:
            if re.match(regexp, line):
                row = rows.get(line)
                if row:
                    row["country"]=country
                else:
                    rows[line]={"country":country}

    data = []
    for row in rows.keys():
        data.append([row, rows[row].get("country", "NaN"), rows[row].get("city", "NaN")])
    print(data)
    
if __name__ == '__main__':
    filename = "./data/headlines.txt"
    parse_data(read_file(filename))